In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
import xgboost as xgb

In [2]:
df = pd.read_csv(r"C:\Users\ALEJANDRO RENDON\Downloads\data_prepd_train_no_pca.csv")
index = df['Customer_ID']
y = df['Credit_Score']
X = df.drop(['Credit_Score','Customer_ID'],axis=1)

In [3]:
gss = GroupShuffleSplit(n_splits=2, test_size=0.3, random_state=42)
for train_index, test_index in gss.split(X, groups=index):
    print("Train:", train_index, "Test:", test_index)
    X_train = X.iloc[train_index, :]
    y_train = y[train_index]
    X_test = X.iloc[test_index, :]
    y_test = y[test_index]

Train: [   16    17    18 ... 99965 99966 99967] Test: [    0     1     2 ... 99997 99998 99999]
Train: [    0     1     2 ... 99989 99990 99991] Test: [    8     9    10 ... 99997 99998 99999]


In [6]:
MLogit = LogisticRegression(class_weight='balanced', random_state=42,max_iter=500, multi_class='multinomial',solver='saga',n_jobs=-1)
MLogit.fit(X_train,y_train)
y_pred = MLogit.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[5878 1350 1549]
 [3631 8083 4333]
 [ 117  804 4255]]
              precision    recall  f1-score   support

           0    0.66970   0.61064   0.63881      9626
           1    0.50371   0.78959   0.61505     10237
           2    0.82206   0.41975   0.55574     10137

    accuracy                        0.60720     30000
   macro avg    0.66516   0.60666   0.60320     30000
weighted avg    0.66454   0.60720   0.60263     30000



In [18]:
param_gridMLogit = { 
    'penalty':['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.0,0.25,0.5,0.75,1.0]
}

In [19]:
CV_MLogit = GridSearchCV(estimator=MLogit, param_grid=param_gridMLogit, cv= 5,scoring='f1_macro')
CV_MLogit.fit(X_train, y_train)
CV_MLogit.best_params_

c:\Users\rendonjimenez.5\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\rendonjimenez.5\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\rendonjimenez.5\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\rendonjimenez.5\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\rendonjimenez.5\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarnin

{'C': 0.25, 'penalty': 'l1'}

In [20]:
MLogit2 = LogisticRegression(class_weight='balanced', random_state=42,max_iter=500, multi_class='multinomial',solver='saga',C=0.25,penalty='l1',n_jobs=16)
MLogit2.fit(X_train,y_train)
y_pred = MLogit2.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[5880 1345 1552]
 [3630 8088 4329]
 [ 116  802 4258]]
              precision    recall  f1-score   support

           0    0.66993   0.61085   0.63903      9626
           1    0.50402   0.79023   0.61548     10235
           2    0.82264   0.41996   0.55606     10139

    accuracy                        0.60753     30000
   macro avg    0.66553   0.60701   0.60352     30000
weighted avg    0.66494   0.60753   0.60295     30000



c:\Users\rendonjimenez.5\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [49]:
CVSMLogit2 = cross_val_score(MLogit2, X_train,y_train, cv=5,scoring='f1_macro')
print([CVSMLogit2,np.average(CVSMLogit2)])

[array([0.60605055, 0.60446279, 0.61005748, 0.59093472, 0.60960632]), 0.6042223731843908]


In [27]:
RCF = RandomForestClassifier(random_state=42,n_jobs=-1, n_estimators=500,class_weight='balanced')
RCF.fit(X_train,y_train)
y_pred = RCF.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[ 5567  2935   275]
 [ 1956 12789  1302]
 [   38  2776  2362]]
              precision    recall  f1-score   support

           0    0.63427   0.73628   0.68148      7561
           1    0.79697   0.69130   0.74038     18500
           2    0.45634   0.59964   0.51827      3939

    accuracy                        0.69060     30000
   macro avg    0.62919   0.67574   0.64671     30000
weighted avg    0.71124   0.69060   0.69637     30000



In [28]:
param_gridRCF = { 
    'max_depth' : np.linspace(10,25,26,dtype=int)
}

In [29]:
CV_RCF = GridSearchCV(estimator=RCF, param_grid=param_gridRCF, cv= 5,scoring='f1_macro',n_jobs=-1)
CV_RCF.fit(X_train, y_train)
CV_RCF.best_params_

{'max_depth': 20}

In [43]:
RCF2 = RandomForestClassifier(random_state=42,n_jobs=-1,max_depth=20,n_estimators=500,class_weight='balanced')
RCF2.fit(X_train,y_train)
y_pred = RCF2.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[ 6161  1846   770]
 [ 2840 10739  2468]
 [   64  1535  3577]]
              precision    recall  f1-score   support

           0    0.70195   0.67965   0.69062      9065
           1    0.66922   0.76055   0.71197     14120
           2    0.69107   0.52487   0.59661      6815

    accuracy                        0.68257     30000
   macro avg    0.68741   0.65502   0.66640     30000
weighted avg    0.68407   0.68257   0.67931     30000



In [44]:
CVSRCF2 = cross_val_score(RCF2, X_train,y_train, cv=5,scoring='f1_macro',n_jobs=-1)
print([CVSRCF2,np.average(CVSRCF2)])

[array([0.66083454, 0.65674361, 0.66590293, 0.6467015 , 0.66174081]), 0.6583846783922173]


In [50]:
LGBM = lgb.LGBMClassifier(random_state=42,n_jobs=-1,n_estimators=500,objective='multiclass',class_weight='balanced',learning_rate=0.01)
LGBM.fit(X_train,y_train)
y_pred = LGBM.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[6685  986 1106]
 [3643 8795 3609]
 [ 128  804 4244]]
              precision    recall  f1-score   support

           0    0.76165   0.63935   0.69516     10456
           1    0.54808   0.83089   0.66048     10585
           2    0.81994   0.47371   0.60050      8959

    accuracy                        0.65747     30000
   macro avg    0.70989   0.64798   0.65205     30000
weighted avg    0.70370   0.65747   0.65465     30000



In [51]:
param_gridLGBM = { 
    'max_depth' : np.linspace(10,25,26,dtype=int)
}

In [52]:
CV_LGBM = GridSearchCV(estimator=LGBM, param_grid=param_gridLGBM, cv= 5,scoring='f1_macro',n_jobs=-1)
CV_LGBM.fit(X_train, y_train)
CV_LGBM.best_params_

{'max_depth': 15}

In [53]:
LGBM2 = lgb.LGBMClassifier(random_state=42,n_jobs=-1,max_depth=15,n_estimators=500,objective='multiclass',class_weight='balanced',learning_rate=0.01)
LGBM2.fit(X_train,y_train)
y_pred = LGBM2.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[6692  975 1110]
 [3637 8793 3617]
 [ 126  796 4254]]
              precision    recall  f1-score   support

           0    0.76245   0.64008   0.69592     10455
           1    0.54795   0.83236   0.66085     10564
           2    0.82187   0.47367   0.60097      8981

    accuracy                        0.65797     30000
   macro avg    0.71076   0.64870   0.65258     30000
weighted avg    0.70471   0.65797   0.65515     30000



In [54]:
CVSLGBM2 = cross_val_score(LGBM2, X_train,y_train, cv=5,scoring='f1_macro')
print([CVSLGBM2,np.average(CVSLGBM2)])

[array([0.64759915, 0.64781597, 0.65982856, 0.63842098, 0.66442574]), 0.6516180801959485]


In [4]:
xgb_cl = xgb.XGBClassifier(random_state =42,n_estimators=500, learning_rate=0.01, tree_method='gpu_hist')
xgb_cl.fit(X_train,y_train)
y_pred = xgb_cl .predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[ 5653  2644   480]
 [ 2118 12045  1884]
 [   73  2019  3084]]
              precision    recall  f1-score   support

           0    0.64407   0.72068   0.68022      7844
           1    0.75061   0.72091   0.73546     16708
           2    0.59583   0.56608   0.58057      5448

    accuracy                        0.69273     30000
   macro avg    0.66350   0.66922   0.66542     30000
weighted avg    0.69464   0.69273   0.69289     30000



In [5]:
param_gridxgb_cl = { 
    'max_depth' : np.linspace(15,20,6,dtype=int)
}

In [8]:
CV_xgb_cl = GridSearchCV(estimator=xgb_cl, param_grid=param_gridxgb_cl, cv= 5,scoring='f1_macro',n_jobs=8)
CV_xgb_cl.fit(X_train, y_train)
CV_xgb_cl.best_params_

In [12]:
xgb_cl2 = xgb.XGBClassifier(random_state =42,n_estimators=500, learning_rate=0.01, tree_method='gpu_hist', max_depth=5)
xgb_cl2.fit(X_train,y_train)
y_pred = xgb_cl2 .predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[ 5564  2739   474]
 [ 1991 12242  1814]
 [   59  2056  3061]]
              precision    recall  f1-score   support

           0    0.63393   0.73076   0.67891      7614
           1    0.76288   0.71855   0.74006     17037
           2    0.59138   0.57226   0.58166      5349

    accuracy                        0.69557     30000
   macro avg    0.66273   0.67386   0.66688     30000
weighted avg    0.69958   0.69557   0.69630     30000



In [13]:
CVSxgb_cl2 = cross_val_score(xgb_cl2, X_train,y_train, cv=5,scoring='f1_macro')
print([CVSxgb_cl2,np.average(CVSxgb_cl2)])

[array([0.65105555, 0.65338573, 0.66846296, 0.6544306 , 0.67753748]), 0.6609744606357798]
